In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [102]:
Ut=U@Asqrt@ROT.T
print(Ut[:3,0],Ut[nn:nn+2,0])

[0.03028344 0.03028344 0.03028344] [0.02581126 0.02581126]


In [31]:
#  15112020 reconfinement
JE,JI=2.0,1.5
a,b=0.6,-0.9#0.3,0.2#
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
print('Rotation:',ROT)
ROT_=ROT.copy()
ROT_[0,:]=ROT[1,:]
ROT_[1,:]=ROT[0,:]
ROT_[:,0]*=(-1)


'''
Generate two expressions in large random matrix
params:
    nn = E/I neuron number
    M/Nrank = low rank structure in EI format
    U/V.    = low rank structyre in SVD format
    M/Nranktilde = low rank EI format for large random matrix
    U/Vtilde = low rank SVD format for large random matrix
    sigvmat: diagonal singular values matrix
    

'''
grandom=0.2
theta = grandom
Rc,R = 0,2
for i in range(2):
    if svr2[i]>grandom:
        Rc+=1
nn=600
sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
for i in range(R):
    sigvmat[i,i]=svr2[i]
    rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
invsigvmat = la.inv(sigvmat)
# print('inverse singular value matrix:',invsigvmat)
# print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

zetau,zetav = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
# zetau= np.random.randn(nn*2,R)/np.sqrt(2*nn)
# zetau[:,1]=zetau[:,0]#np.sqrt(0.8)*zetau[:,0]+np.sqrt(0.2)*np.squeeze(np.random.randn(nn*2,1))/np.sqrt(2*nn)
zetav = zetau # correlation between random terms
zetaM,zetaN = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
zetaM_,zetaN_ = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
# generate rank-2 matrix P without random entries
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
# use SVD to decompose the low rank matrix
lvec,svr,rvech=la.svd(P)
rvec=rvech.T 
# check the singular value
# print(svr,svr2)
if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
    print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
Asqrt=np.eye(R)#alphamat.copy()
invAsqrt = np.zeros_like(Asqrt)
for i in range(R):
    if alphamat[i,i]>0.0:
        Asqrt[i,i]=np.sqrt(alphamat[i,i])
        invAsqrt[i,i]=1.0/Asqrt[i,i]
    else:
        Asqrt[i,i]=1e-3
        invAsqrt[i,i]=1.0/Asqrt[i,i]
randomterm=np.zeros((R,R))
for i in range(R):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
# EI format
withrand,wrank = 1,1
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=1,-1
N2[:nn],N2[nn:]=a/nn,b/nn
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
NT = np.random.randn(nn*2,R)
NT = NT.T 
if R>1:
    Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
else:
    Mrank[:,0],Nrank[:,0]=M1,N1
zetaMtilde = np.sqrt(2*nn)*zetaM*grandom@invsigvmat@la.inv(Asqrt)@ROT.T
zetaNtilde = 1.0/np.sqrt(2*nn)*zetaN*grandom@la.inv(Asqrt)@ROT.T
'''orthogonal'''
c=1.0
zetaMtilde_ = np.sqrt(2*nn)*zetaM_*grandom@invsigvmat@la.inv(Asqrt)@ROT.T#_#
zetaNtilde_ = 1.0/np.sqrt(2*nn)*zetaN_*grandom@la.inv(Asqrt)@ROT.T#_#
# zetaMtilde_ = np.sqrt(2*nn)*zetaM_*grandom@invsigvmat@invAsqrt@ROT_#
# zetaNtilde_ = 1.0/np.sqrt(2*nn)*zetaN_*grandom@invAsqrt@ROT_#
Mranktilde=(Mrank+zetaMtilde_*c+zetaMtilde*(1-c))@ROT@Asqrt
Nranktilde=(Nrank+zetaNtilde_*c+zetaNtilde*(1-c))@ROT@Asqrt@invsigvmat@(sigvmat+grandom**2*invsigvmat)
Nranktildeh = Nranktilde.T
U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
U,V=lvec[:,:R],rvec[:,:R]
Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@randomterm,wrank*V@Asqrt+withrand*zetav@randomterm

# add random part in, Xtilde = X+P
X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Xtilde=X+P
Utdnum,sigvtdnum,Vtdnumh=la.svd(Xtilde)
Utdnum2,sigvtdnum2,Vtdnumh2=Utdnum[:,:Rc],sigvtdnum[:Rc],Vtdnumh[:Rc,:]
Vtdnum2=Vtdnumh2.T
for i in range(Rc):
    Vtdnum2[:,i]*=sigvtdnum2[i]
Xtilde_num=Vtdnum2.T@Utdnum2
if Rc>1:
    Xtilde_svdrec = (sigvmat+(grandom**2)*invsigvmat)@Vtilde.T@Utilde#Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
    Xtilde_eirec  = Nranktildeh@Mranktilde#Nranktildeh@Mranktilde#/grandom**2
    Xtilde_low = Nrank.T@Mrank
else:
    Xtilde_svdrec = np.reshape(Vtilde[:,0],(1,2*nn))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Utilde[:,0],(nn*2,1))#@#/grandom**2
    Xtilde_eirec  = np.reshape(Nranktilde[:,0],(1,2*nn))@np.reshape(Mranktilde[:,0],(2*nn,1))#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2
    Xtilde_low = np.reshape(Nrank[:,0],(1,nn*2))@np.reshape(Mrank[:,0],(nn*2,1))

# if Rc>1:
#     Xtilde_svdrec = Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
#     Xtilde_eirec  = Mranktilde@Nranktildeh#Nranktildeh@Mranktilde#/grandom**2
#     Xtilde_low = Mrank@Nrank.T
# else:
#     Xtilde_svdrec = np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
#     Xtilde_eirec  = np.reshape(Mranktilde[:,0],(2*nn,1))@np.reshape(Nranktilde[:,0],(1,2*nn))
#     Xtilde_low = np.reshape(Mrank[:,0],(nn*2,1))@np.reshape(Nrank[:,0],(1,nn*2))

# test two expressions, using singular values
_,sigvxtilde,_=la.svd(Xtilde_num)#Xtilde)
_,sigveitilde,_=la.svd(Xtilde_eirec)
_,sigvsvdtilde,_=la.svd(Xtilde_svdrec)

_,sigvlow,_=la.svd(Xtilde_low)

print('compare the singular values from')
print('original:',sigvxtilde[:Rc])
print('svd:',sigvsvdtilde[:Rc])
print('ei:',sigveitilde[:Rc])#Xtilde_eirec)#
print('low:',sigvlow[:Rc])




Rotation: [[-0.92350317  0.38359079]
 [-0.38359079 -0.92350317]]
compare the singular values from
original: [3.54652139 0.4901756 ]
svd: [3.5189723  0.53913121]
ei: [3.50171094 0.49694515]
low: [3.82339652 0.47078559]


In [32]:
print(U[:2,0],U[nn:nn+2,0],U[:2,1],U[nn:nn+2,1])

[-0.03773255 -0.03773255] [-0.01558593 -0.01558593] [-0.01558593 -0.01558593] [0.03773255 0.03773255]


In [95]:
ROT_=ROT.copy()
ROT_[0,:]=ROT[1,:]
ROT_[1,:]=ROT[0,:]
ROT_[:,0]*=(-1)
print(ROT_@ROT)
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
# zetaMtilde,zetaNtilde=-zetaMtilde/np.sqrt(2*nn),-zetaNtilde*np.sqrt(2*nn)
zetaMtilde_ = np.sqrt(2*nn)*zetaM*grandom@invsigvmat@la.inv(Asqrt)@ROT_
zetaNtilde_ = 1.0/np.sqrt(2*nn)*zetaN*grandom@la.inv(Asqrt)@ROT_
zetaMtilde_,zetaNtilde_=-zetaMtilde_/np.sqrt(2*nn),-zetaNtilde_*np.sqrt(2*nn)
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Mrank[:,1]+zetaMtilde[:,1]),c='pink',alpha=0.5)
ax1.scatter(-(Mrank[:,0]+zetaMtilde_[:,0]),-(Mrank[:,1]+zetaMtilde_[:,1]),c='green',alpha=0.5)
ax1.set_aspect('equal')

ax1.set_xlabel(r'$\tilde{M}^{(1)}$',fontsize=14)
ax1.set_ylabel(r'$\tilde{M}^{(2)}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b)+'\n $\sigma_1=$'+str(format(svr2[0],'.1f'))+r',$\rho_1=$'+str(format(rhosigv[0,0],'.1f'))+',$\overline{\sigma}=$'+str(grandom),fontsize=16)

[[-5.55111512e-17  1.00000000e+00]
 [-1.00000000e+00  5.55111512e-17]]


Text(0.5, 1.0, '$J_E,\\ J_I=$2.9,1.5, $a, b=$0.6,-0.9\n $\\sigma_1=$4.8,$\\rho_1=$4.9,$\\overline{\\sigma}=$0.5')

In [48]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
# zetaMtilde,zetaNtilde=-zetaMtilde/np.sqrt(2*nn),-zetaNtilde*np.sqrt(2*nn)

for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(121)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Mrank[:,1]+zetaMtilde[:,1]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax2=fig.add_subplot(122)
ax2.scatter(-(Nrank[:,0]+zetaNtilde[:,0]),-(Nrank[:,1]+zetaNtilde[:,1]),c='green',alpha=0.5)
ax2.set_aspect('equal')

fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(121)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Nrank[:,0]+zetaNtilde[:,0]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax2=fig.add_subplot(122)
ax2.scatter(-(Mrank[:,1]+zetaMtilde[:,1]),-(Nrank[:,1]+zetaNtilde[:,1]),c='green',alpha=0.5)
ax2.set_aspect('equal')

fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(121)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Nrank[:,1]+zetaNtilde[:,1]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax2=fig.add_subplot(122)
ax2.scatter(-(Mrank[:,1]+zetaMtilde[:,1]),-(Nrank[:,0]+zetaNtilde[:,0]),c='green',alpha=0.5)
ax2.set_aspect('equal')


In [34]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
# zetaMtilde,zetaNtilde=-zetaMtilde/np.sqrt(2*nn),-zetaNtilde*np.sqrt(2*nn)

for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-(Mrank[:,RANKP]+zetaMtilde[:,RANKP]),-(Nrank[:,RANKP]+zetaNtilde[:,RANKP]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)
RANKP=1
fig=plt.figure(figsize=(6,6))
ax2=fig.add_subplot(111)
ax2.scatter(-(Mrank[:,RANKP]+zetaMtilde[:,RANKP]),-(Nrank[:,RANKP]+zetaNtilde[:,RANKP]),c='pink',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+(grandom**2)*invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

RANKP=1
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax2.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax2.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)


In [344]:
#  15112020 reconfinement
JE,JI=2.9,1.5
a,b=0.6,-0.9#0.3,0.2#
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
print('Rotation:',ROT)

'''
Generate two expressions in large random matrix
params:
    nn = E/I neuron number
    M/Nrank = low rank structure in EI format
    U/V.    = low rank structyre in SVD format
    M/Nranktilde = low rank EI format for large random matrix
    U/Vtilde = low rank SVD format for large random matrix
    sigvmat: diagonal singular values matrix
    

'''
grandom=0.5
theta = grandom
Rc,R = 0,2
for i in range(2):
    if svr2[i]>grandom:
        Rc+=1
nn=600
sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
for i in range(R):
    sigvmat[i,i]=svr2[i]
invsigvmat = la.inv(sigvmat)

zetau,zetav = grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn),grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn)
zetaM,zetaN = grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn),grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn)
# generate rank-2 matrix P without random entries
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
# use SVD to decompose the low rank matrix
lvec,svr,rvech=la.svd(P)
rvec=rvech.T 
# check the singular value
# print(svr,svr2)
if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
    print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
Asqrt=np.eye(R)#alphamat.copy()
for i in range(R):
    if alphamat[i,i]>0.0:
        Asqrt[i,i]=np.sqrt(alphamat[i,i])
    else:
        Asqrt[i,i]=0.0
randomterm=np.zeros((R,R))
for i in range(R):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)

U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
U,V=lvec[:,:R],rvec[:,:R]

# add random part in, Xtilde = X+P
X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Xtilde=X+P
# test two expressions, using singular values
lvectd,sigvxtilde,rvechtd=la.svd(Xtilde)
Utilde,Vtilde = lvectd[:,:R],rvechtd[:R,:]
Vtilde = Vtilde.T



Rotation: [[-0.95863948  0.28462317]
 [-0.28462317 -0.95863948]]


In [345]:
print('original sigv:',svr[:2],'; critical value:',grandom)
print('random sigv num:',sigvxtilde[:2],';')
theosigv = svr[:2]+grandom**2/svr[:2]
print('theoretical sigv:',theosigv)


original sigv: [4.81194992 0.71073059] ; critical value: 0.5
random sigv num: [4.86061557 1.0633878 ] ;
theoretical sigv: [4.86390391 1.06248131]


In [346]:
print('overlap bewteen singular vectors, num:',np.sum(U[:,0]*Utilde[:,0])**2,';right ',np.sum(V[:,0]*Vtilde[:,0])**2)
print('theoretical:',Asqrt[0,0]**2)
print('overlap bewteen singular vectors, num:',np.sum(U[:,1]*Utilde[:,1])**2,';right ',np.sum(V[:,1]*Vtilde[:,1])**2)
print('theoretical:',Asqrt[1,1]**2)

overlap bewteen singular vectors, num: 0.9888076931786727 ;right  0.9888560118383901
theoretical: 0.9892031315204817
overlap bewteen singular vectors, num: 0.5356105817001876 ;right  0.5493838003262217
theoretical: 0.5050857094904272


## $\mathbf{\tilde{M}},\mathbf{\tilde{N}}$ expression

In [84]:
# change to CLASS?
def StatisticsLRandomMat(Connparams,Netparams,):
    JE,JI,a,b,g = Connparams
    nn,withrand,wrank,thetatilde = Netparams
    nn = int(nn)
    # #  15112020 reconfinement
    # JE,JI=2.0,1.5#2.9,1.5
    # a,b=0.725,-0.775#0.3,0.2#
    # # with random components or not
    # withrand,wrank,thetatilde = 1.0,1.0,0.6*np.pi

    # calculate the rotation matrix in rank-2 case (reduced), then extend to large 
    # generate perturbed matrix
    J2=np.zeros((2,2))
    J2[:,0],J2[:,1]=JE,-JI
    J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
    J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
    lvec2,svr2,rvech2=la.svd(J2)
    # print('svr2:',svr2)
    M2 = np.array([[1,1],[1,-1]])
    ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
    # print('sv2:',svr2)

    '''
    Generate two expressions in large random matrix
    params:
        nn = E/I neuron number
        M/Nrank = low rank structure in EI format
        U/V.    = low rank structyre in SVD format
        M/Nranktilde = low rank EI format for large random matrix
        U/Vtilde = low rank SVD format for large random matrix
        sigvmat: diagonal singular values matrix


    '''
    grandom=(svr2[1]*g)#svr2[1]*0.70#
    print('g threshold, upper:',2*grandom)
    theta = grandom
    Rc,R = 0,2
    for i in range(2):
        if svr2[i]>grandom:
            Rc+=1
#     nn=600
    sigvmat,rhosigv = np.zeros((Rc,Rc)),np.zeros((Rc,Rc))
    for i in range(Rc):
        sigvmat[i,i]=svr2[i]
        rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
    invsigvmat = la.inv(sigvmat)
    # print('inverse singular value matrix:',invsigvmat)
    # print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

    zetau,zetav = np.random.randn(nn*2,Rc)/np.sqrt(2*nn),np.random.randn(nn*2,Rc)/np.sqrt(2*nn)
    nx,ny = np.random.randn(nn*2,Rc)/np.sqrt(2*nn),np.random.randn(nn*2,Rc)/np.sqrt(2*nn)
    zetau=ny.copy()
    zetav=nx.copy()
    # generate rank-2 matrix P without random entries
    P=np.zeros((nn*2,nn*2))
    P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
    P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
    P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
    # use SVD to decompose the low rank matrix
    lvec,svr,rvech=la.svd(P)
    rvec=rvech.T 
    # check the singular value
    # print(svr,svr2)
    if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
        print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

    '''
        Analytical calculation
    '''
    # diagonal matrix D^-1 and (I-D^-2)^1/2
    # (I-D^-2)^1/2
    alphamat= np.eye(Rc)-(grandom**2)*la.matrix_power(invsigvmat,2)
    Asqrt=np.eye(Rc)
    for i in range(Rc):
        if alphamat[i,i]>0.0:
            Asqrt[i,i]=np.sqrt(alphamat[i,i])
        else:
            Asqrt[i,i]=0.0
            print('AI')
    # print('Asqrt:',Asqrt)
    # D^-1
    randomterm=np.zeros((Rc,Rc))
    for i in range(Rc):
        randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)

    # Analytically calculate the singular values after transformation(perturbation)
    Alyrho = sigvmat.copy()
    for i in range(Rc):
        Alyrho[i,i]=Alyrho[i,i]+grandom**2/Alyrho[i,i]
        
    # U,V low-rank
    U,V=np.zeros((nn*2,Rc)),np.zeros((nn*2,Rc))
    U,V=lvec[:,:Rc],rvec[:,:Rc]
    # U,V overline (after transformation)
    # U,V zetau,v overline (after transformation)
    Uoverline,Voverline = wrank*U@Asqrt,wrank*V@Asqrt
    zetauoverline,zetavoverline = withrand*zetau@randomterm,withrand*zetav@randomterm
    # U,V tilde (used for reconstruction)
    Utilde,Vtilde = Uoverline+zetauoverline,Voverline+zetavoverline
    
    if Rc>1:
        # randomly choose a rotation matrix for U,Vtilde and M,Ntilde
        Otilde = np.zeros((Rc,Rc))
        Otilde = np.array([[np.cos(thetatilde),-np.sin(thetatilde)],[np.sin(thetatilde),np.cos(thetatilde)]])
        # clustring mean after rotation M,N overline
        # from Uoverline and Voverrline (adding analytically calculated singular value)
        Moverline,Noverline = Uoverline@Otilde,Voverline@Alyrho@Otilde

        # change the rotation matrix Otilde
        # the 1st mode of column vector in Moverline is averaging-mode
        # Uup = Uoverline[0,:],Ubottom=Uoverline[-1,:]
        gamma = np.arctan(U[-1,0]/U[0,0])
        thetatildeAvg = gamma+np.pi/4.0#(U[-1,0]-U[0,0])/(U[0,1]-U[-1,1])
        # tgOtilde = (Uoverline[0,0]-Uoverline[-1,0])/(Uoverline[-1,1]-Uoverline[0,1])
        # thetatildeAvg=-np.arctan(tgOtilde)
        OtildeAvg = np.array([[np.cos(thetatildeAvg),-np.sin(thetatildeAvg)],[np.sin(thetatildeAvg),np.cos(thetatildeAvg)]])
        print('rotation:',OtildeAvg)

        # random/perturbation components for M and N, zetamoverline,zetanoverline
        zetamoverline,zetanoverline = zetauoverline@Otilde,zetavoverline@Alyrho@Otilde
        # final expression for M,N tilde
        Mtilde,Ntilde=Moverline+zetamoverline,Noverline+zetanoverline
        Ntildeh = Ntilde.T
    elif Rc==1:
        Moverline,Noverline = Uoverline,Voverline@Alyrho

        # change the rotation matrix Otilde
        # the 1st mode of column vector in Moverline is averaging-mode
        # Uup = Uoverline[0,:],Ubottom=Uoverline[-1,:]
        tgOtilde = 1.0
        OtildeAvg = 1.0

        # random/perturbation components for M and N, zetamoverline,zetanoverline
        zetamoverline,zetanoverline = zetauoverline,zetavoverline@Alyrho
        # final expression for M,N tilde
        Mtilde,Ntilde=Moverline+zetamoverline,Noverline+zetanoverline
        Ntildeh = Ntilde.T       

    '''
        Numerical simulation
    '''
    # add random part in, Xtilde = X+P
    X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
    Xtilde=X+P
    Utildenum,sigvxtilde,Vtildenumh=la.svd(Xtilde)
    # the first 2 structural connectivity
    Utildenum,Vtildenum =Utildenum[:,:Rc], Vtildenumh.T
    Vtildenum=Vtildenum[:,:Rc]
    # calculate the mean values of left and right singular vectors U,V numerical overline
    Uoverlinenum,Voverlinenum=Utildenum.copy(),Vtildenum.copy()
    for i in range(Rc):
        Uoverlinenum[:nn,i],Voverlinenum[:nn,i]=np.mean(Uoverlinenum[:nn,i]),np.mean(Voverlinenum[:nn,i])
        Uoverlinenum[nn:,i],Voverlinenum[nn:,i]=np.mean(Uoverlinenum[nn:,i]),np.mean(Voverlinenum[nn:,i])
    # dissect the random components, zetau,v numerical tilde
    zetauoverlinenum,zetavoverlinenum=Utildenum-Uoverlinenum,Vtildenum-Voverlinenum
    if Rc>1:
        MoverlineAvg=Uoverlinenum@OtildeAvg
    elif Rc==1:
        MoverlineAvg=Uoverlinenum


    '''
        properties
    '''
    Ovlzetauvoverline = zetavoverlinenum.T@zetauoverlinenum
    # Overlapuv = Vtildenum.T@Utildenum#sigvmat@Overlapuv@sigvmat/(grandom**2)
    OvlUV = V.T@U
    # calculate the random terms in Utilde and Vtilde
    if Rc>1:
        Xtilde_svdrec = Utilde@Alyrho@Vtilde.T#/grandom**2
        Xtilde_eirec  = Mtilde@Ntildeh#Nranktildeh@Mranktilde#/grandom**2
    else:
        Xtilde_svdrec = Utilde@Alyrho@Vtilde.T#np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
        Xtilde_eirec  = Mtilde@Ntildeh#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2


    # test eigenvalues
    eigv,eigvec=la.eig(Xtilde)
    print('Eigenvalues:',eigv[:R+1])
    _,sigveitilde,_=la.svd(Xtilde_eirec)
    _,sigvsvdtilde,_=la.svd(Xtilde_svdrec)
    NumLRandMat = {'LRandMat':Xtilde,
                   'RandComp':X,
                   'LowRComp':P,
                   'ltilde':Utildenum,
                   'rtilde':Vtildenum,
                   'singularv':sigvxtilde,
                   'lvecmean':Uoverlinenum,
                   'rvecmean':Voverlinenum,
                   'zetalvec':zetauoverlinenum,
                   'zetarvec':zetavoverlinenum
        
    }
    TheosvdLRandMat = {'LRandMat':Xtilde_svdrec,
                   'ltilde':Utilde,
                   'rtilde':Vtilde,
                   'singularv':sigvsvdtilde,
                   'lvecmean':Uoverline,
                   'rvecmean':Voverline,
                   'zetalvec':zetauoverline,
                   'zetarvec':zetavoverline,
                   'Theosingularv':Alyrho 
        
    }
    TheoeiLRandMat = {'LRandMat':Xtilde_eirec,
                   'ltilde':Mtilde,
                   'rtilde':Ntilde,
                   'singularv':sigveitilde,
                   'lvecmean':Moverline,
                   'rvecmean':Noverline,
                   'zetalvec':zetamoverline,
                   'zetarvec':zetanoverline
        
    }
    return (NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandom,Asqrt)

## The Statistics of Large Random Matrix
* **Numerical Solution**
* **Theoretical/Analytical Solution**

In [85]:
JE,JI,a,b,g=2.9,1.5,0.725,-0.775,0.5#2.9,1.5,0.3,0.2#
nn,withrand,wrank,thetatilde = 800,1.0,1.0,0.96*np.pi
Connparams=np.array([JE,JI,a,b,g])
Netparams =np.array([nn,withrand,wrank,thetatilde])
NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandom,Asqrt=StatisticsLRandomMat(Connparams,Netparams,)

g threshold, upper: 0.48019737908368637
rotation: [[ 0.29577984 -0.95525613]
 [ 0.95525613  0.29577984]]
Eigenvalues: [1.45038147+0.47013053j 1.45038147-0.47013053j 0.17113209+0.17120147j]


In [86]:
thetagamma = np.arctan(U[-1,0]/U[0,0])
print('tan gamma for U:',U[-1,0]/U[0,0])
thetatilde = thetagamma+np.pi/4.0
Otilde = np.array([[np.cos(thetatilde),-np.sin(thetatilde)],[np.sin(thetatilde),np.cos(thetatilde)]])
print('rotation:',Otilde)
# Uoverline
g = grandom
a1,a2=np.sqrt(1-g**2/(svr2[0])**2),np.sqrt(1-g**2/(svr2[1])**2)
print('Asqwrt',a1,a2)
Ubar=np.zeros((2,2))
Ubar[0,0],Ubar[1,0]=a1*U[0,0],a1*U[-1,0]
Ubar[0,1],Ubar[1,1]=a2*U[0,1],a2*U[-1,1]
print('U overline here:',Ubar)
print(TheosvdLRandMat['lvecmean'][:2,0],TheosvdLRandMat['lvecmean'][nn:nn+2,0])
print(TheosvdLRandMat['lvecmean'][:2,1],TheosvdLRandMat['lvecmean'][nn:nn+2,1])

print('M overline meaning:',Ubar@Otilde)

tan gamma for U: 0.5271441433980699
rotation: [[ 0.29577984 -0.95525613]
 [ 0.95525613  0.29577984]]
Asqwrt 0.9987643895377316 0.8660254037844386
U overline here: [[-0.03123725 -0.01427808]
 [-0.01646654  0.02708572]]
[-0.03123725 -0.03123725] [-0.01646654 -0.01646654]
[-0.01427808 -0.01427808] [0.02708572 0.02708572]
M overline meaning: [[-0.02287857  0.02561641]
 [ 0.02100333  0.02374117]]


In [87]:
# the overlap between random component *NUMERICAL*
print('RANDOM COMPONENT:')
print(Ovlzetauvoverline)
print('LOWRANK COMPONENT:')
print(OvlUV)

# compare the singular values: numerical, theoretical solutions
print('Singular Values, NUMERICAL:')
print(NumLRandMat['singularv'][:3])
print('NUMERICAL (RECONSTRUCTED)')
print('SVD:',TheosvdLRandMat['singularv'][:3])
print('EI:',TheoeiLRandMat['singularv'][:3])
print('THEORETICAL:')
print(TheosvdLRandMat['Theosingularv'])

# check the rotation matrix for averaging-mode
print('Averaging-mode after Appropriate Rotation:',MoverlineAvg[:3,0],MoverlineAvg[nn:nn+3:,0])
print('2nd mode after Appropriate Rotation:',MoverlineAvg[:3,1],MoverlineAvg[nn:nn+3:,1])

RANDOM COMPONENT:
[[-4.05120493e-05 -5.35585435e-04]
 [ 8.88368610e-04  6.54451551e-03]]
LOWRANK COMPONENT:
[[ 0.5459806   0.83779782]
 [-0.83779782  0.5459806 ]]
Singular Values, NUMERICAL:
[4.84771239 0.59897025 0.47625917]
NUMERICAL (RECONSTRUCTED)
SVD: [4.83576613e+00 5.91134556e-01 2.56991680e-15]
EI: [4.83576613e+00 5.91134556e-01 1.92813133e-15]
THEORETICAL:
[[4.8432786  0.        ]
 [0.         0.60024672]]
Averaging-mode after Appropriate Rotation: [-0.02283416 -0.02283416 -0.02283416] [0.02104807 0.02104807 0.02104807]
2nd mode after Appropriate Rotation: [0.02564534 0.02564534 0.02564534] [0.02372202 0.02372202 0.02372202]


In [236]:
# fig=plt.figure()
# ax=fig.add_subplot(311)
# ax.hist(sigvxtilde, 100, density=True, facecolor='g', alpha=0.75)
# ax.plot(sigveitilde[:2], np.array([0,0]),'r*')
# ax.set_aspect('auto')
# ax.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b)+'\n $\sigma_1=$'+str(format(svr2[0],'.1f'))+r',$g=$'+str(grandom),fontsize=12)

# ax1=fig.add_subplot(312)
# ax1.hist(sigvxtilde, 100, density=True, facecolor='g', alpha=0.75)
# ax1.plot(sigveitilde[:2], np.array([0,0]),'r*')
# ax1.set_aspect('auto')
# ax1.set_title(r',$g=$'+str(grandom),fontsize=12)

# ax2=fig.add_subplot(313)
ax2.hist(sigvxtilde, 100, density=True, facecolor='g', alpha=0.75)
ax2.plot(sigveitilde[:2], np.array([0,0]),'r*')
ax2.set_aspect('auto')
ax2.set_title(r',$g=$'+str(grandom),fontsize=12)
ax2.set_xlabel(r'$\sigma_{\rho}$',fontsize=14)
ax1.set_ylabel(r'$P(\rho_i)$',fontsize=14)

Text(148.31944444444443, 0.5, '$P(\\rho_i)$')

In [80]:
#  15112020 reconfinement
JE,JI=2.9,1.5
a,b=0.6,-0.9#0.3,0.2#
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
print('sv2:',svr2)

'''
Generate two expressions in large random matrix
params:
    nn = E/I neuron number
    M/Nrank = low rank structure in EI format
    U/V.    = low rank structyre in SVD format
    M/Nranktilde = low rank EI format for large random matrix
    U/Vtilde = low rank SVD format for large random matrix
    sigvmat: diagonal singular values matrix
    

'''
grandom=3.2
theta = grandom
Rc,R = 0,2
for i in range(2):
    if svr2[i]>grandom:
        Rc+=1
nn=600
sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
for i in range(R):
    sigvmat[i,i]=svr2[i]
    rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
invsigvmat = la.inv(sigvmat)
# print('inverse singular value matrix:',invsigvmat)
# print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

zetau,zetav = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
nx,ny = np.random.randn(nn*2,R)/1,np.random.randn(nn*2,R)/nn
zetau=nx.copy()
zetav=ny.copy()
zetaM,zetaN = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
# generate rank-2 matrix P without random entries
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
# use SVD to decompose the low rank matrix
lvec,svr,rvech=la.svd(P)
rvec=rvech.T 
# check the singular value
# print(svr,svr2)
if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
    print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
Asqrt=np.eye(R)#alphamat.copy()
for i in range(R):
    if alphamat[i,i]>0.0:
        Asqrt[i,i]=np.sqrt(alphamat[i,i])
    else:
        Asqrt[i,i]=0.0
print('determined:',Asqrt)
randomterm=np.zeros((R,R))
for i in range(R):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
    print('random:',randomterm[i,i])
# EI format
withrand,wrank = 1.0,1
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=1,-1
N2[:nn],N2[nn:]=a/nn,b/nn
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
NT = np.random.randn(nn*2,R)
NT = NT.T 
if R>1:
    Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
else:
    Mrank[:,0],Nrank[:,0]=M1,N1
Mranktilde = wrank*Mrank@ROT@Asqrt+withrand*zetaM@randomterm*np.sqrt(2*nn)
Nranktildeh = (sigvmat+(grandom**2)*invsigvmat)@(wrank*Asqrt@invsigvmat@ROT.T@Nrank.T+withrand*randomterm@zetaN.T/np.sqrt(2*nn))#+(sigvmat+invsigvmat)@invsigvmat@zetav.T
Nranktilde=Nranktildeh.T 

U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
U,V=lvec[:,:R],rvec[:,:R]
Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@randomterm,wrank*V@Asqrt+withrand*zetav@randomterm

# add random part in, Xtilde = X+P
X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Xtilde=X+P
if Rc>1:
    Xtilde_svdrec = Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
    Xtilde_eirec  = Mranktilde@Nranktildeh#Nranktildeh@Mranktilde#/grandom**2
else:
    Xtilde_svdrec = np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
    Xtilde_eirec  = np.reshape(Mranktilde[:,0],(2*nn,1))@np.reshape(Nranktilde[:,0],(1,2*nn))#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2

# test two expressions, using singular values
_,sigvxtilde,_=la.svd(Xtilde)
_,sigveitilde,_=la.svd(Xtilde_eirec)
_,sigvsvdtilde,_=la.svd(Xtilde_svdrec)

print('compare the singular values from')
print('original:',sigvxtilde[:3])
print('svd:',sigvsvdtilde[:3])
print('ei:',sigveitilde[:3])#Xtilde_eirec)#




sv2: [4.81194992 0.71073059]
determined: [[0.74683349 0.        ]
 [0.         0.        ]]
random: 0.6650110772920023
random: 1.0
compare the singular values from
original: [6.88564361 6.40377966 6.34042435]
svd: [3.88273602e+00 2.00382196e-15 1.66031086e-15]
ei: [6.74688013e+00 3.53291755e-15 3.49687526e-15]


** Large Random Matrix Has Two Outliers **

In [23]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)
RANKP=1
fig=plt.figure(figsize=(6,6))
ax2=fig.add_subplot(111)
ax2.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+(grandom**2)*invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

RANKP=1
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax2.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax2.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)


In [25]:
ax1.set_xlabel(r'$\tilde{M}^{(1)}$',fontsize=14)
ax1.set_ylabel(r'$\tilde{N}^{(1)}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b)+'\n $\sigma_1=$'+str(format(svr2[0],'.1f'))+r',$\rho_1=$'+str(format(rhosigv[0,0],'.1f'))+',$\overline{\sigma}=$'+str(grandom),fontsize=16)

ax2.set_xlabel(r'$\tilde{M}^{(2)}$',fontsize=14)
ax2.set_ylabel(r'$\tilde{N}^{(2)}$',fontsize=14)
ax2.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b)+'\n $\sigma_2=$'+str(format(svr2[1],'.1f'))+r',$\rho_2=$'+str(format(rhosigv[1,1],'.1f'))+',$\overline{\sigma}=$'+str(grandom),fontsize=16)

Text(0.5, 1.0, '$J_E,\\ J_I=$2.9,1.5, $a, b=$0.6,-0.9\n $\\sigma_2=$0.7,$\\rho_2=$1.1,$\\overline{\\sigma}=$0.5')

In [5]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+(grandom**2)*invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

# ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
# ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)



In [355]:
print('U original:',U[:2,:],U[nn:nn+2,:])
print('sqrt',np.sqrt(1-1/svr[:2]**2)-1.0)

U original: [[-0.0358899  -0.01945717]
 [-0.0358899  -0.01945717]] [[-0.01945717  0.0358899 ]
 [-0.01945717  0.0358899 ]]
sqrt [-0.02183206         nan]


In [7]:
ax1.scatter(-Mrank[:,1],-Nrank[:,1],c='cyan',alpha=0.5)
ax1.set_xlabel(r'$\tilde{M}^{(1)}$',fontsize=14)
ax1.set_ylabel(r'$\tilde{N}^{(1)}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b),fontsize=16)

Text(0.5, 1.0, '$J_E,\\ J_I=$2.9,1.5, $a, b=$0.6,-0.9')

## Figures: Gaussian Distribution

In [80]:
## Gaussian distributions for Utilde,U and Vtilde,V

# numeerically obtain Utildenum, Vtildenum
Utildenum,rho,Vtildenumh=la.svd(Xtilde)
Utildenum = Utildenum[:,:R]
Vtildenumh = Vtildenumh[:R,:]
Vtildenum = Vtildenumh.T 
Udiffnum,Vdiffnum=Utildenum-U@Asqrt,Vtildenum-V@Asqrt
Udiffnum[:,1]+=2*U[:,1]*Asqrt[1,1]
Vdiffnum[:,1]+=2*V[:,1]*Asqrt[1,1]

Nspace = 1000
# left singular matrix
maxstd = np.std(np.reshape(Udiffnum,(1,-1)))
fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(Udiffnum[:,0], 100, density=True, alpha=0.5)
stdtest=np.std(Udiffnum[:,0])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[0]*np.sqrt(2*nn))
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
ax1.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(1)}$')
ax1.set_xlim([-5*maxstd,5*maxstd])
ax1.legend()
n, bins, patches = ax2.hist(Udiffnum[:,1], 100, density=True, alpha=0.5)
stdtest=np.std(Udiffnum[:,1])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[1]*np.sqrt(2*nn))
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
# disttest/=np.sum(disttest)
ax2.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(2)}$')
ax2.set_xlim([-5*maxstd,5*maxstd])
ax2.set_xlabel('Entries')
ax2.legend()

# n, bins, patches = ax2.hist(residrvec[:,1], 100, density=True, alpha=0.5)
# stdtest=np.std(residrvec[:,1])
# xxx=np.linspace(-3*stdtest,3*stdtest,1000)
# disttest = np.exp(-(xxx/stdtest)**2/2)/np.sqrt(2*np.pi*stdtest**2)
# # disttest/=np.sum(disttest)
# ax2.plot(xxx,disttest*nn*2/1000)


In [86]:
# numeerically obtain Mtildenum, Ntildenum
Utildenum,rho,Vtildenumh=la.svd(Xtilde)
Utildenum = Utildenum[:,:R]
Vtildenumh = Vtildenumh[:R,:]
Vtildenum = Vtildenumh.T 
Mtilde_t=Utildenum-np.random.randn(nn*2,R)@la.matrix_power(invsigvmat,3)/np.sqrt(2*nn)
Mtildenum=Mtilde_t*np.sqrt(2*nn)@la.inv(Asqrt)
Mdiffnum =Mtildenum-Mrank@ROT
# Udiffnum[:,1]+=2*U[:,1]*Asqrt[1,1]
# Vdiffnum[:,1]+=2*V[:,1]*Asqrt[1,1]

Nspace = 1000
# left singular matrix
maxstd = np.std(np.reshape(Mdiffnum,(1,-1)))
fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(Mdiffnum[:,0], 100, density=True, alpha=0.5)
stdtest=np.std(Mdiffnum[:,0])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[0]*1)
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
ax1.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(1)}$')
ax1.set_xlim([-5*maxstd,5*maxstd])
ax1.legend()
n, bins, patches = ax2.hist(Mdiffnum[:,1], 100, density=True, alpha=0.5)
stdtest=np.std(Mdiffnum[:,1])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[1])
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
# disttest/=np.sum(disttest)
ax2.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(2)}$')
ax2.set_xlim([-5*maxstd,5*maxstd])
ax2.set_xlabel('Entries')
ax2.legend()

## Testing Two Representations

In [14]:
#  15112020 reconfinement
JE,JI=1.0,1.45
a,b=0.40,0.50#0.3,0.2#
nvalues,niterations=20,50
aseries = np.linspace(0.0,1.0,nvalues)
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
grandom=0.5
Singularvalue=2*grandom*np.ones((3,2,nvalues,niterations))
for iter in range(niterations):
    for idxa, a in enumerate(aseries):
        J2=np.zeros((2,2))
        J2[:,0],J2[:,1]=JE,-JI
        J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
        J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
        lvec2,svr2,rvech2=la.svd(J2)
        M2 = np.array([[1,1],[1,-1]])
        ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
        # print('Rotation:',ROT)

        '''
        Generate two expressions in large random matrix
        params:
            nn = E/I neuron number
            M/Nrank = low rank structure in EI format
            U/V.    = low rank structyre in SVD format
            M/Nranktilde = low rank EI format for large random matrix
            U/Vtilde = low rank SVD format for large random matrix
            sigvmat: diagonal singular values matrix
            

        '''
        theta = grandom
        Rc,R = 0,2
        for i in range(2):
            if svr2[i]>grandom:
                Rc+=1
        nn=600
        sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
        for i in range(R):
            sigvmat[i,i]=svr2[i]
            rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
        invsigvmat = la.inv(sigvmat)
        # print('inverse singular value matrix:',invsigvmat)
        # print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

        zetau,zetav = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
        zetaM,zetaN = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
        # generate rank-2 matrix P without random entries
        P=np.zeros((nn*2,nn*2))
        P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
        P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
        P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
        # use SVD to decompose the low rank matrix
        lvec,svr,rvech=la.svd(P)
        rvec=rvech.T 
        # check the singular value
        # print(svr,svr2)
        if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
            print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

        alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
        Asqrt=np.eye(R)#alphamat.copy()
        for i in range(R):
            if alphamat[i,i]>0.0:
                Asqrt[i,i]=np.sqrt(alphamat[i,i])
            else:
                Asqrt[i,i]=0.0
        randomterm=np.zeros((R,R))
        for i in range(R):
            randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
        # EI format
        withrand,wrank = 1,1
        M1,N1=np.ones(2*nn),np.ones(2*nn)
        N1[:nn],N1[nn:]=JE/nn,-JI/nn
        M2,N2=M1.copy(),N1.copy()
        M2[:nn],M2[nn:]=1,-1
        N2[:nn],N2[nn:]=a/nn,b/nn
        Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
        NT = np.random.randn(nn*2,R)
        NT = NT.T 
        if R>1:
            Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
        else:
            Mrank[:,0],Nrank[:,0]=M1,N1
        Mranktilde = wrank*Mrank@ROT@Asqrt+withrand*zetaM@randomterm*np.sqrt(2*nn)
        Nranktildeh = (sigvmat+(grandom**2)*invsigvmat)@(wrank*Asqrt@invsigvmat@ROT.T@Nrank.T+withrand*randomterm@zetaN.T/np.sqrt(2*nn))#+(sigvmat+invsigvmat)@invsigvmat@zetav.T
        Nranktilde=Nranktildeh.T 

        U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
        U,V=lvec[:,:R],rvec[:,:R]
        Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@randomterm,wrank*V@Asqrt+withrand*zetav@randomterm

        # add random part in, Xtilde = X+P
        X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
        Xtilde=X+P
        if Rc>1:
            Xtilde_svdrec = Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
            Xtilde_eirec  = Mranktilde@Nranktildeh#Nranktildeh@Mranktilde#/grandom**2
        else:
            Xtilde_svdrec = np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
            Xtilde_eirec  = np.reshape(Mranktilde[:,0],(2*nn,1))@np.reshape(Nranktilde[:,0],(1,2*nn))#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2
        
        # test two expressions, using singular values
        _,sigvxtilde,_=la.svd(Xtilde)
        _,sigveitilde,_=la.svd(Xtilde_eirec)
        _,sigvsvdtilde,_=la.svd(Xtilde_svdrec)

        if Rc<2:
            Singularvalue[:,0,idxa,iter]=np.array([sigvxtilde[0],sigveitilde[0],sigvsvdtilde[0]])
        else:
            Singularvalue[0,:,idxa,iter]=sigvxtilde[:Rc]
            Singularvalue[1,:,idxa,iter]=sigveitilde[:Rc]
            Singularvalue[2,:,idxa,iter]=sigvsvdtilde[:Rc]

np.save('Singularvalue18Nov_1.npy', Singularvalue)  
# print('compare the singular values from')
# print('original:',sigvxtilde[:2])
# print('svd:',sigvsvdtilde[:2])
# print('ei:',sigveitilde[:2])




In [322]:
data = np.load('Singularvalue17Nov.npy')
Singularvalues = data
print(np.shape(Singularvalues))
ntype,nsigv,nseries,niterations=np.shape(Singularvalues)
# calculate std
meanstdseries = np.zeros((ntype,nsigv,nseries,2))
for it in range(ntype):
    for isv in range(nsigv):
        for isr in range(nseries):
            meanstdseries[it,isv,isr,0]=np.mean(np.squeeze(Singularvalues[it,isv,isr,:]))
            meanstdseries[it,isv,isr,1]=np.std(np.squeeze(Singularvalues[it,isv,isr,:]))


(3, 2, 20, 50)


In [15]:
Singularvalues=Singularvalue.copy()
ntype,nsigv,nseries,niterations=np.shape(Singularvalues)
# calculate std
meanstdseries = np.zeros((ntype,nsigv,nseries,2))
for it in range(ntype):
    for isv in range(nsigv):
        for isr in range(nseries):
            meanstdseries[it,isv,isr,0]=np.mean(np.squeeze(Singularvalues[it,isv,isr,:]))
            meanstdseries[it,isv,isr,1]=np.std(np.squeeze(Singularvalues[it,isv,isr,:]))
aseries = np.linspace(0.0,1.0,nseries)
fig=plt.figure()
ax1=fig.add_subplot(111)
# for iter in range
ax1.plot(aseries,meanstdseries[0,0,:,0],'r',label=r'numerical $\tilde{\boldsymbol{X}}$')
plt.fill_between(aseries, meanstdseries[0,0,:,0]-meanstdseries[0,0,:,1], meanstdseries[0,0,:,0]+meanstdseries[0,0,:,1],
facecolor="orange", # The fill color
color='orange',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[1,0,:,0],'b',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
plt.fill_between(aseries, meanstdseries[1,0,:,0]-meanstdseries[1,0,:,1], meanstdseries[1,0,:,0]+meanstdseries[1,0,:,1],
facecolor="orange", # The fill color
color='blue',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[2,0,:,0],'g',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')
plt.fill_between(aseries, meanstdseries[2,0,:,0]-meanstdseries[2,0,:,1], meanstdseries[2,0,:,0]+meanstdseries[2,0,:,1],
facecolor="orange", # The fill color
color='green',       # The outline color
alpha=0.1)          # Transparency of the fill

ax1.plot(aseries,meanstdseries[0,1,:,0],'r--',label=r'numerical $\tilde{\boldsymbol{X}}$')
plt.fill_between(aseries, meanstdseries[0,1,:,0]-meanstdseries[0,1,:,1], meanstdseries[0,1,:,0]+meanstdseries[0,1,:,1],
facecolor="orange", # The fill color
color='orange',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[1,1,:,0],'b--',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
plt.fill_between(aseries, meanstdseries[1,1,:,0]-meanstdseries[1,1,:,1], meanstdseries[1,1,:,0]+meanstdseries[1,1,:,1],
facecolor="orange", # The fill color
color='blue',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[2,1,:,0],'g--',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')
plt.fill_between(aseries, meanstdseries[2,1,:,0]-meanstdseries[2,1,:,1], meanstdseries[2,1,:,0]+meanstdseries[2,1,:,1],
facecolor="orange", # The fill color
color='green',       # The outline color
alpha=0.1)          # Transparency of the fill
# ax1.plot(aseries,Singularvalue[1,0,:],'b',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
# ax1.plot(aseries,Singularvalue[2,0,:],'g',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')

# ax1.plot(aseries,Singularvalue[0,1,:],'r--',label=r'numerical $\tilde{\boldsymbol{X}}$')
# ax1.plot(aseries,Singularvalue[1,1,:],'b--',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
# ax1.plot(aseries,Singularvalue[2,1,:],'g--',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')

In [16]:
ax1.set_xlabel(r'$a$',fontsize=14)
ax1.set_ylabel(r'$\rho_{i}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $b=$'+str(b)+'\n $\overline{\sigma}=$'+str(grandom),fontsize=16)



Text(0.5, 1.0, '$J_E,\\ J_I=$1.0,1.45, $b=$0.5\n $\\overline{\\sigma}=$0.5')

In [119]:
# np.save('Singularvalue16Nov.npy', Singularvalue)  
fig=plt.figure()
ax1=fig.add_subplot(111)
# plt.fill_between(x, y1, y2,
# facecolor="orange", # The fill color
# color='blue',       # The outline color
# alpha=0.2)          # Transparency of the fill
for iter in range
ax1.plot(aseries,Singularvalue[0,0,:],'r',label=r'numerical $\tilde{\boldsymbol{X}}$')
ax1.plot(aseries,Singularvalue[1,0,:],'b',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
ax1.plot(aseries,Singularvalue[2,0,:],'g',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')

ax1.plot(aseries,Singularvalue[0,1,:],'r--',label=r'numerical $\tilde{\boldsymbol{X}}$')
ax1.plot(aseries,Singularvalue[1,1,:],'b--',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
ax1.plot(aseries,Singularvalue[2,1,:],'g--',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')


In [49]:
MRT = Mrank@ROT.T/np.sqrt(2*nn)
print('compare MRT and U:',U[:+2,:],MRT[:+2,:]) 
print('compare MRT and U:',U[nn:nn+2,:],MRT[nn:nn+2,:]) 


compare MRT and U: [[-0.02693818 -0.03067574]
 [-0.02693818 -0.03067574]] [[-0.02693818 -0.03067574]
 [-0.02693818 -0.03067574]]
compare MRT and U: [[-0.03067574  0.02693818]
 [-0.03067574  0.02693818]] [[-0.03067574  0.02693818]
 [-0.03067574  0.02693818]]


In [50]:
NRTINV = Nrank@ROT.T@invsigvmat*np.sqrt(2*nn)
print('compare MRT and U:',V[:+2,:],NRTINV[:+2,:]) 
print('compare MRT and U:',V[nn:nn+2,:],NRTINV[nn:nn+2,:]) 

compare MRT and U: [[-0.03536305 -0.02039906]
 [-0.03536305 -0.02039906]] [[-0.03536305 -0.02039906]
 [-0.03536305 -0.02039906]]
compare MRT and U: [[ 0.02039906 -0.03536305]
 [ 0.02039906 -0.03536305]] [[ 0.02039906 -0.03536305]
 [ 0.02039906 -0.03536305]]


In [57]:
print('compare MRT and U:',Utilde[:+2,:],Mranktilde[:+2,:]) 
print('compare MRT and U:',Utilde[nn:nn+2,:],Mranktilde[nn:nn+2,:]) 

compare MRT and U: [[-0.03397748 -0.03438452]
 [-0.02605377 -0.02787703]] [[-0.03397748 -0.03438452]
 [-0.02605377 -0.02787703]]
compare MRT and U: [[-0.03207857  0.03029732]
 [-0.04216421  0.00877207]] [[-0.03207857  0.03029732]
 [-0.04216421  0.00877207]]


In [58]:
print('compare MRT and U:',Vtilde[:+2,:],Nranktilde[:+2,:]) 
print('compare MRT and U:',Vtilde[nn:nn+2,:],Nranktilde[nn:nn+2,:]) 

compare MRT and U: [[-0.02189407 -0.01804848]
 [-0.04999292 -0.01212403]] [[ 0.01241503 -0.00075827]
 [-0.01568382  0.00516618]]
compare MRT and U: [[ 0.00946424 -0.07184279]
 [ 0.01846816 -0.02033009]] [[-0.01032685 -0.04186912]
 [-0.00132293  0.00964357]]
